<a href="https://colab.research.google.com/github/luizleroy/predictive_analytics/blob/develop/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction to Predictive Analysis with Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/luizleroy/predictive_analytics/blob/develop/new.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab! I will get a quick introduction to the Analysis with Colab and Python to Predictive Analysis.

Colab is a Python development environment that runs in the browser using Google Cloud.

In [ ]:
print("Agora, vamos na língua de Camões mesmo, hehehe. Pois, po principal objetivo aqui é criar insumos, 'pedaços' de código mesmo, para utilizar em projetos de P&D.")

Agora, vamos na língua de Camões mesmo, hehehe. Pois, po principal objetivo aqui é criar insumos, 'pedaços' de código mesmo, para utilizar em projetos de P&D.


# Imports

In [ ]:
import numpy as np
import pandas as pd
import json


# Code

In [46]:
data = pd.read_json('sample_data/anscombe.json')
X = data['X']
Y = data['Y']
X1 = X[0:11]
X2 = X[11:22]
X3 = X[22:33]
X4 = X[33:42]
columns = ['I','II', 'III', 'IV']
data_para_treinamento = pd.DataFrame(columns=columns)

print(data_para_treinamento)

Empty DataFrame
Columns: [um, dois, tres, quatro]
Index: []


In [53]:
# Z = data.transpose()
# print(data)
# print("transpondo...")
# print(Z)

   Series   X      Y
0       I  10   8.04
1       I   8   6.95
2       I  13   7.58
3       I   9   8.81
4       I  11   8.33
5       I  14   9.96
6       I   6   7.24
7       I   4   4.26
8       I  12  10.84
9       I   7   4.81
10      I   5   5.68
11     II  10   9.14
12     II   8   8.14
13     II  13   8.74
14     II   9   8.77
15     II  11   9.26
16     II  14   8.10
17     II   6   6.13
18     II   4   3.10
19     II  12   9.13
20     II   7   7.26
21     II   5   4.74
22    III  10   7.46
23    III   8   6.77
24    III  13  12.74
25    III   9   7.11
26    III  11   7.81
27    III  14   8.84
28    III   6   6.08
29    III   4   5.39
30    III  12   8.15
31    III   7   6.42
32    III   5   5.73
33     IV   8   6.58
34     IV   8   5.76
35     IV   8   7.71
36     IV   8   8.84
37     IV   8   8.47
38     IV   8   7.04
39     IV   8   5.25
40     IV  19  12.50
41     IV   8   5.56
42     IV   8   7.91
43     IV   8   6.89
transpondo...
          0     1     2     3     4     5